In [1]:
from bs4 import BeautifulSoup
from requests import get
import pandas as pd

In [2]:
url = r'https://math.stackexchange.com/questions'
response = get(url)
html_soup = BeautifulSoup(response.text, 'html.parser')